In [3]:
import matplotlib.pyplot as plt
from matplotlib.sankey import Sankey
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [4]:
wd = "/Users/hubertcheung/Documents/GitHub/property_marriage"
df = pd.read_csv(wd+"/data/ind&econ.csv")

df1 = df.loc[df['marry'] == 1, ['pid','year','owntype','marry_y']]

/var/folders/nc/fq1fpth513jgq0tl1k0gs3c00000gn/T/ipykernel_24617/3780663437.py:2: DtypeWarning: Columns (3,8,9,16,19,26,35,46,47,48,51,52,53,54,55,67,68,71,72,75,76,78,82,87,88,89,90,94,95,96,97,98,99,100,104,106,107) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(wd+"/data/ind&econ.csv")


In [5]:
pid2010 = df1[df1['year'] == 2010]['pid'].tolist()
pid2012 = df1[df1['year'] == 2012]['pid'].tolist()
pid2014 = df1[df1['year'] == 2014]['pid'].tolist()
pid2016 = df1[df1['year'] == 2016]['pid'].tolist()
pid2018 = df1[df1['year'] == 2018]['pid'].tolist()

pidcommon = list(set(pid2010) & set(pid2012) & set(pid2014) & set(pid2016) & set(pid2018))

df_refined = df1[df1['pid'].isin(pidcommon)]
df_refined['year'].value_counts()
df_wide = df_refined.pivot(index='pid', columns='year', values='owntype')
df_wide.reset_index(inplace=True)
df_wide = df_wide.fillna("")

In [6]:
df_wide.to_stata(wd+"/data/sankey.dta")

/var/folders/nc/fq1fpth513jgq0tl1k0gs3c00000gn/T/ipykernel_24617/3408557903.py:1: InvalidColumnName: 
Not all pandas column names were valid Stata variable names.
The following replacements have been made:

    2010   ->   _2010
    2012   ->   _2012
    2014   ->   _2014
    2016   ->   _2016
    2018   ->   _2018

If this is not what you expect, please make sure you have Stata-compliant
column names in your DataFrame (strings only, max 32 characters, only
alphanumerics and underscores, no Stata reserved words)

  df_wide.to_stata(wd+"/data/sankey.dta")


In [54]:
sources_s = pd.concat([df_wide[2010], df_wide[2012], df_wide[2014], df_wide[2016]])
targets_s = pd.concat([df_wide[2012], df_wide[2014], df_wide[2016], df_wide[2018]])
pids = pd.concat([df_wide['pid'], df_wide['pid'], df_wide['pid'], df_wide['pid']])
year = df_refined.loc[df_refined['year']<=2016,['year']]

df_new = pd.DataFrame({'source': sources_s, 'target': targets_s, 'pid': pids})
df_new = df_new.reset_index()
year = year['year'].reset_index()
df_new = pd.concat([df_new, year['year']], axis=1)

In [68]:
color_link_all = ['#FFFF00', '#1CE6FF', '#FF34FF', '#FF4A46',
             '#008941', '#006FA6', '#A30059','#FFDBE5', '#7A4900', 
             '#0000A6', '#63FFAC', '#B79762', '#004D43', '#8FB0FF',
             '#997D87', '#5A0007', '#809693', '#FEFFE6', '#1B4400', 
             '#4FC601', '#3B5DFF', '#4A3B53', '#FF2F80', '#61615A',
             '#BA0900', '#6B7900', '#00C2A0', '#FFAA92', '#FF90C9',
             '#B903AA', '#D16100', '#DDEFFF', '#000035', '#7B4F4B',                
             '#A1C299', '#300018', '#0AA6D8', '#013349', '#00846F',
             '#372101', '#FFB500', '#C2FFED', '#A079BF', '#CC0744',
             '#C0B9B2', '#C2FF99', '#001E09', '#00489C', '#6F0062', 
             '#0CBD66', '#EEC3FF', '#456D75', '#B77B68', '#7A87A1',
             '#788D66', '#885578', '#FAD09F', '#FF8A9A', '#D157A0',
             '#BEC459', '#456648', '#0086ED', '#886F4C', '#34362D', 
             '#B4A8BD', '#00A6AA', '#452C2C', '#636375', '#A3C8C9', 
             '#FF913F', '#938A81', '#575329', '#00FECF', '#B05B6F',
             '#8CD0FF', '#3B9700', '#04F757', '#C8A1A1', '#1E6E00',
             '#7900D7', '#A77500', '#6367A9', '#A05837', '#6B002C',
             '#772600', '#D790FF', '#9B9700', '#549E79', '#FFF69F', 
             '#201625', '#72418F', '#BC23FF', '#99ADC0', '#3A2465',
             '#922329', '#5B4534', '#FDE8DC', '#404E55', '#0089A3',
             '#CB7E98', '#A4E804', '#324E72', '#6A3A4C'
             ]
color_link = color_link_all[0:16]*4

In [23]:
nodes_origin = ["self","spouse","co","other"]
years = ["2010","2012","2014","2016","2018"]

nodes = [j+i for i in years for j in nodes_origin]

['self2010',
 'spouse2010',
 'co2010',
 'other2010',
 'self2012',
 'spouse2012',
 'co2012',
 'other2012',
 'self2014',
 'spouse2014',
 'co2014',
 'other2014',
 'self2016',
 'spouse2016',
 'co2016',
 'other2016',
 'self2018',
 'spouse2018',
 'co2018',
 'other2018']

In [73]:
data = df_new.to_dict("records")
links = []

# Sankey diagram
for point in data:
    link = {'source' : point['source'], 'target' : point['target'], 'year' : point['year']}
    links.append(link)

sources = []
targets = []
tuples = []
values = []


for i in range(len(nodes)):
    for j in range(len(nodes)):
        for link in links:
            if link['source']+str(link['year']) == nodes[i] and link['target']+str(link['year']+2) == nodes[j]:
                if (nodes[i],nodes[j]) not in tuples:
                    tuples.append((nodes[i],nodes[j]))
                    values.append(1)
                else:
                    values[tuples.index((nodes[i],nodes[j]))] += 1
                    
sources = [nodes.index(t[0]) for t in tuples]
targets = [nodes.index(t[1]) for t in tuples]

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=nodes,
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      color = color_link
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color = color_link
    ))])

# Customize the layout
fig.update_layout(title_text="Transition from Year 1 to Year 2",
                  font=dict(size=12, color='black'))

# Display the Sankey diagram
fig.show()

In [59]:
data = df_new.to_dict("records")
links = []

# Sankey diagram
for point in data:
    link = {'source' : point['source'], 'target' : point['target'], 'year' : point['year']}
    links.append(link)

sources = []
targets = []
tuples = []
values = []


for i in range(len(nodes)):
    for j in range(len(nodes)):
        for link in links:
            if link['source']+str(2010) == nodes[i] and link['target']+str(2012) == nodes[j]:
                if (nodes[i],nodes[j]) not in tuples:
                    tuples.append((nodes[i],nodes[j]))
                    values.append(1)
                else:
                    values[tuples.index((nodes[i],nodes[j]))] += 1
                    
sources = [nodes.index(t[0]) for t in tuples]
targets = [nodes.index(t[1]) for t in tuples]

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=nodes,
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      color = color_link
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color = color_link
    ))])

# Customize the layout
fig.update_layout(title_text="Transition from Year 1 to Year 2",
                  font=dict(size=12, color='black'))

# Display the Sankey diagram
fig.show()

In [35]:
import plotly.graph_objects as go

# Sample data
data = [
    {'id': 1, 'type_1': 'A', 'type_2': 'B'},
    {'id': 2, 'type_1': 'A', 'type_2': 'C'},
    {'id': 3, 'type_1': 'B', 'type_2': 'B'},
    {'id': 4, 'type_1': 'C', 'type_2': 'B'},
    {'id': 5, 'type_1': 'C', 'type_2': 'C'},
    {'id': 6, 'type_1': 'A', 'type_2': 'C'}
]

# Create nodes and links
nodes = []
links = []

for row in data:
    links.append({'source': row['type_1'], 'target': row['type_2'], 'value': 1})

    if row['type_1'] not in nodes:
        nodes.append(row['type_1'])
    if row['type_2'] not in nodes:
        nodes.append(row['type_2'])

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        label=nodes,
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      color = "blue"
    ),
    link=dict(
        source=[nodes.index(link['source']) for link in links],
        target=[nodes.index(link['target']) for link in links],
        value=[link['value'] for link in links]
    ))])

# Customize the layout
fig.update_layout(title_text="Transition from Year 1 to Year 2",
                  font=dict(size=12, color='black'))

# Display the Sankey diagram
fig.show()

print(links)
print(nodes)
print([link['source'] for link in links])

[{'source': 'A', 'target': 'B', 'value': 1}, {'source': 'A', 'target': 'C', 'value': 1}, {'source': 'B', 'target': 'B', 'value': 1}, {'source': 'C', 'target': 'B', 'value': 1}, {'source': 'C', 'target': 'C', 'value': 1}, {'source': 'A', 'target': 'C', 'value': 1}]
['A', 'B', 'C']
['A', 'A', 'B', 'C', 'C', 'A']


In [1]:
import pandas as pd
from pySankey import sankey

pd.options.display.max_rows = 8
df = pd.read_csv(
    'fruits.txt', sep=' ', names=['true', 'predicted']
)
colorDict = {
    'apple':'#f71b1b',
    'blueberry':'#1b7ef7',
    'banana':'#f3f71b',
    'lime':'#12e23f',
    'orange':'#f78c1b'
}
sankey(
    df['true'], df['predicted'], aspect=20, colorDict=colorDict,
    fontsize=12, figureName="fruit"
)
# Result is in "fruit.png"

TypeError: 'module' object is not callable

year,pid,2010,2012,2014,2016,2018
0,110009102,self,co,self,self,self
1,110009106,other,co,spouse,spouse,spouse
2,110026102,NaN,NaN,NaN,other,NaN
3,110060101,self,self,self,self,self
4,110107102,self,self,self,self,self
...,...,...,...,...,...,...
1397,621818101,spouse,spouse,spouse,spouse,spouse
1398,621819101,NaN,NaN,NaN,NaN,self
1399,621820101,self,self,self,self,self
1400,621822102,self,self,NaN,self,NaN
